# Numerical exercise QM

Numerical solution of TDSE etc.

In [ ]:
'''Import necessary packages'''
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Set common figure parameters:
newparams = {'axes.labelsize': 11, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (15, 12),
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,}
plt.rcParams.update(newparams)

In [ ]:
'''Parameters and constants'''
h_bar = 1
m = 1
k_0 = 20
L = 20
N = 1000
omega = h_bar * k_0**2 / (2*m) # Obs. is this correct?? Jø nø se pa, håper det.
E = h_bar * omega # Is this correct?? Håper det også :^)
dx = L/(N-1)

In [ ]:
x = np.linspace(0, L, N) # Spacing should be almost dx

In [ ]:
def timestep(psi_im, psi_re, v, dt):
    """Performs one timestep of size dt
    Input:
        psi_im : Psi_im(x, t)
        psi_re : Psi_re(x, t + dt/2)
    Returns:
        psi_im : Psi_im(x, t + dt)
        psi_re : Psi_re(x, t + dt)
    """
    
    assert psi_im[0] == psi_im[-1] == psi_re[0] == psi_re[-1] == 0, "Psi should be zero for x = 0 and x = L"
    # Find psi_im(t+dt) given psi_im(t) and psi_re(t + dt/2)
    psi_im[1:-1] -= dt*(v[1:-1]*psi_re[1:-1]/h_bar - (psi_re[2:] - 2*psi_re[1:-1] + psi_re[:-2])*h_bar/(2*m*dx**2))
    
    # Find psi_re(t+ 3*dt/2) given psi_re(t+dt/2) and psi_im(t+dt) (remember, psi_im is now at t = t+dt)
    psi_re[1:-1] += dt*(v[1:-1]*psi_im[1:-1]/h_bar - (psi_im[2:] - 2*psi_im[1:-1] + psi_im[:-2])*h_bar/(2*m*dx**2))
    
    return psi_im, psi_re

## Problem 1

We start by calculating the initial wave function, normalising it, and plotting the real and imaginary parts. We also plot the initial probability distribution for the particles position given by $\mid{\Psi(x,t)}\mid^2$.

In [ ]:
''' Problem 1'''
x_s = 5 # Start position
x_f = L/2 + x_s # final position, later we will let the wave propagate til it reaches here, ellerno
sigma_x = 1.5

# Normalization
C = 1/np.sqrt(
    np.sum(np.exp(-(x-x_s)**2/(sigma_x**2)))*dx
)

v = np.zeros(N) # Zero potential
dt = 0.1 * h_bar / (h_bar**2 /(2*m*dx**2) + np.max(v)) # Timestep, much smaller than ..., see (12)

'''Parameters for propagation'''
v_g = h_bar*k_0/m # group velocity
sim_time = L/(2*v_g) # simulation time
num_iter = int(sim_time/dt) # number of iterations

# Arrays for storing psis at different points in time, need this later
psi_re = np.zeros([num_iter, N])
psi_im = np.zeros([num_iter, N])
# see (8)
psi_re[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.cos(k_0*x[1:-1] - omega*dt/2)
psi_im[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.sin(k_0*x[1:-1])

In [ ]:
''' Plot start state, real and imaginary'''
plt.plot(x, psi_re[0], label="Initial Re")
plt.plot(x, psi_im[0], label="Initial Im")
plt.title('Initial wave function')
plt.legend()
simulated_time = dt*num_iter
print(f"Simulated time: {simulated_time}s")

In [ ]:
'''Initial probability density'''
initial_prob_density = np.square(np.absolute(psi_re[0] + psi_im[0]*1j))
plt.plot(x, initial_prob_density, label='')
plt.title('Probability density in the initial state')
plt.show()

## Problem 2

We now let the wave packet propagate from its starting position $x_s = 5$ to $x_f = 15$.

In [ ]:
def simulate(v, x_s, sigma_x, num_iter, psi_real, psi_imag):
    '''
    Cool docstring
    '''
    C = 1/np.sqrt(
    np.sum(np.exp(-(x-x_s)**2/(sigma_x**2)))*dx
    )
    
    psi_real[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.cos(k_0*x[1:-1] - omega*dt/2)
    psi_imag[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.sin(k_0*x[1:-1])

    for i in range(num_iter-1):
        psi_imag[i+1], psi_real[i+1] = timestep(psi_imag[i], psi_real[i], v, dt) #todo: pass more sensible dt
        
    return psi_real, psi_imag

In [ ]:
'''This cell runs the simulation'''
psi_re, psi_imag = simulate(v, x_s, sigma_x, num_iter, psi_re, psi_im)

In [ ]:
# Plot start and final state
plt.plot(x, psi_re[0], label="Initial Re")
plt.plot(x, psi_re[-1], label="Final Re")
plt.legend()
simulated_time = dt*num_iter
print(f"Simulated time: {simulated_time}s")

In [ ]:
'''Animation'''
from matplotlib import animation
from IPython.display import HTML
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams.update({'animation.html':'html5', 'savefig.dpi': 100})
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot('111', projection='3d')
def init_anim():
    #ax.plot(x, psi_re[0], psi_im[0])
    global line
    line, = ax.plot(x, psi_re[0], psi_re[1])
    #ax.set_xlim([0, L])
    #ax.set_ylim([-1, 1])
    #ax.set_zlim([-1, 1])
    
def animate(y):
    global line
    line.set_data(x, y[0])
    line.set_3d_properties(y[1])
    
anim = animation.FuncAnimation(fig, animate, init_func=init_anim, frames=zip(psi_re[::100], psi_im[::100]), interval=60)
plt.close(anim._fig)
HTML(anim.to_html5_video())

In [ ]:
from matplotlib import animation
from IPython.display import HTML
plt.rcParams.update({'animation.html':'html5', 'savefig.dpi': 100})

def init_anim():
    global ax, line1, line2
    line1, = ax.plot([], [])
    line2, = ax.plot([], [])
    ax.set_xlim([0, L])
    ax.set_ylim([-1, 1])
    
    
def animate(y):
    global ax, line
    line1.set_data(x, y[0])
    line2.set_data(x, y[1])
    
    
fig, ax = plt.subplots()
anim = animation.FuncAnimation(fig, animate, init_func=init_anim, frames=zip(psi_re[::100], psi_im[::100]), interval=60)
plt.close(anim._fig)
HTML(anim.to_html5_video())

In [ ]:
'''Simulation for more sigma_x's'''
sigma_xs = [0.5, 1.0, 2.0]

psi_re_sgms = np.zeros([len(sigma_xs), num_iter, N])
psi_im_sgms = np.zeros([len(sigma_xs), num_iter, N])

for j in range(len(sigma_xs)):
    psi_re_sgms[j], psi_im_sgms[j] = simulate(v, x_s, sigma_xs[j], num_iter, psi_re, psi_im)

In [ ]:
''' Plot start and final state for the different sigmas'''
for i in range(len(sigma_xs)):
    plt.plot(x, psi_re_sgms[i][0], label="Initial Re")
    plt.plot(x, psi_re_sgms[i][-1], label="Final Re")
    plt.legend()
    plt.title('sigma_x = ' + str(sigma_xs[i]))
    plt.show()
    simulated_time = dt*num_iter
    print(f"Simulated time: {simulated_time}s")

## Introducing a barrier

In [ ]:
'''Defines a barrier'''
l = L/50 # barrier width
v_0 =  3*E/4 # barrier height
v_barr = v_0*np.heaviside(x*np.heaviside(-x +L/2 +l/2, 1) -L/2+l/2, 0)

In [ ]:
'''Runs simulation with the barrier potential'''
psi_re, psi_im = simulate(v_barr, x_s, sigma_x, num_iter, psi_re, psi_im)

In [ ]:
'''Plot start and final state'''
plt.plot(x, psi_re[0], label="Initial")
plt.plot(x, psi_re[-1], label="FInal")
plt.legend()
simulated_time = dt*num_iter
print(f"Simulated time: {simulated_time}s")

In [ ]:
'''Animation'''
fig, ax = plt.subplots()
ax.plot(x, v_barr)
anim = animation.FuncAnimation(fig, animate, init_func=init_anim, frames=zip(psi_re[::100], psi_im[::100]), interval=60)
plt.close(anim._fig)
HTML(anim.to_html5_video())

In [ ]:
'''Calculates P(right) and P(left), probability of being right and left of the barrier'''
mid_i = int(N/2) # Index of center

P_right = np.sum((psi_re**2+psi_im**2)[[0, -1], mid_i:], axis=1)*dx
P_left = np.sum((psi_re**2+psi_im**2)[[0, -1], :mid_i], axis=1)*dx

print(f"P_right: {P_right}\nP_left: {P_left}\nSum: {P_right + P_left}")

In [ ]:
print(f"Probability of transmission: {P_right[1]}")

In [ ]:
'''Calculates transmission probability as a function of barrier height'''
v = E*np.heaviside(x*np.heaviside(-x +L/2 +l/2, 1) -L/2+l/2, 0)
fraction = np.linspace(0, 3/2, 50)
P_trans = []

for f in fraction:
    psi_re, psi_im = simulate(f*v_barr, x_s, sigma_x, num_iter, psi_re, psi_im)
    P_trans.append(np.sum((psi_re**2+psi_im**2)[-1, mid_i:])*dx)

In [ ]:
plt.plot(fraction, P_trans)

In [ ]:
# Find transmission probability as a function of barrier width
v_0 =  9*E/10 # barrier height

widths = np.linspace(0, L/20, 50)
P_trans = []

#psi_re = np.zeros([num_iter, N])
#psi_im = np.zeros([num_iter, N])
# see (8)
#psi_re[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.cos(k_0*x[1:-1] - omega*dt/2)
#psi_im[0, 1:-1] = C*np.exp(-(x[1:-1]-x_s)**2/(2*sigma_x**2)) * np.sin(k_0*x[1:-1])

for l in widths:
    v = v_0*np.heaviside(x*np.heaviside(-x +L/2 +l/2, 1) -L/2+l/2, 0)
    psi_re, psi_im = simulate(f*v_barr, x_s, sigma_x, num_iter, psi_re, psi_im)
    P_trans.append(np.sum((psi_re**2+psi_im**2)[-1, mid_i:])*dx)
plt.plot(widths, P_trans)